In [3]:
# Loading required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import string #for punctuation
from pycontractions  import Contractions
from emoji import replace_emoji
import re
import pattern
from pattern.en import lemma
from keybert import KeyBERT
import json
from nltk.tokenize import sent_tokenize, word_tokenize

## Reading Reviews

In [4]:
# The comments.csv file is create in the "reviews_keyphrases.ipynb" file
# I am using the prior checkpoint files and building things on top of that
df_reviews = pd.read_csv('./data_after_processing/reviews_related_data/comments_with_sentiments.csv')

In [5]:
# https://medium.com/@lukei_3514/dealing-with-contractions-in-nlp-d6174300876b
# Pycontractions model
cont = Contractions('GoogleNews-vectors-negative300.bin')
cont.load_models()

In [6]:
def sent_tokenize_with_indices(text):
    # Sentence Tokenizer
    sentences = sent_tokenize(text)
    offset = 0
    output = []
    for sentence in sentences:
        offset = text.find(sentence, offset)
        # Append the start & end positions of the tokenized sentence retrived from the review comment processed
        output.append({
            'sentence': sentence,
            'start': offset,
            'end': offset + len(sentence)
        })
        offset += len(sentence)
    return output

In [8]:
df_reviews['cleaned_comments'] = df_reviews['comments'] \
    .apply(lambda x: x.lower()) \
    .apply(lambda x: x.replace('<br/>', '')) \
    .apply(replace_emoji, args=(''))\
    .apply(lambda x : re.sub('[^\x00-\x7F]+' , "'", x))

In [9]:
uncontracted = list(cont.expand_texts(df_reviews['cleaned_comments'].to_list(), precise=True))

In [10]:
df_reviews['expanded_clean_comments'] = uncontracted


In [11]:
df_reviews['sentences'] = df_reviews['expanded_clean_comments'].apply(sent_tokenize_with_indices)

In [16]:
df_reviews.to_parquet('./data_after_processing/reviews_related_data/comments_sentences_offset.parquet', index=False)